In [2]:
import re
import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from datetime import datetime
from PIL import Image

In [3]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Pratham
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Pratham
[nltk_data]     jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:

# Download the NLTK tagger if necessary
nltk.download("averaged_perceptron_tagger")

# Define the list of keywords to extract
keyword_list = ["eyebrows", "eyes", "hair", "head", "lips", "mustache", "nose"]

# Define the path to the image folder
image_folder = "../speech-to-text/images"

# Define the path to the feedback file
feedback_file = "../speech-to-text/feedback.txt"

#adjectives list
adjective_list = list(wn.all_eng_synsets(wn.ADJ))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pratham jain\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [76]:
# adjective_list

In [99]:
# Open the "op.txt" file and read the latest line
with open("op.txt", "r") as file:
    line = file.readlines()[-1].strip()

# Define a regular expression to match adjectives
adj_regex = re.compile(r"\b(?:" + "|".join([adj.name() for adj in adjective_list]) + r")\b", re.IGNORECASE)

# Tokenize the sentence into words and tags
tagged_words = nltk.pos_tag(word_tokenize(line))

# Initialize empty lists for adjectives and keywords
adjectives = []
keywords = []

# Loop through each word and tag
for i, (word, tag) in enumerate(tagged_words):
    # Check if the word is a keyword
    if word.lower() in keyword_list:
        # Check if there is an adjective before this keyword
        if i > 0 and tagged_words[i-1][1] in ["JJ", "JJR", "JJS"]:
            # Add the adjective and keyword to their respective lists
            adjectives.append(tagged_words[i-1][0])
            keywords.append(word)


print(keywords)

#In this code, we first define the list of keywords to extract. We then download the NLTK tagger if necessary and tokenize the sentence into words and tags using the pos_tag function.

#Next, we initialize empty lists for adjectives and keywords. We then loop through each word and tag, and check if the word is a keyword. If the word is a keyword, we check if there is an adjective before this keyword. If there is, we add the adjective and keyword to their respective lists.

#After that, we combine the adjectives and keywords into phrases, just as in the previous code.

#Finally, we define the path to the image folder and loop through each phrase. For each phrase, we get the keyword from the phrase and define the path to the subfolder. We then search for the image file in the subfolder using the os.listdir function. If the image file is found, we combine the subfolder path and the image file name and display the image using the PIL library.

['hair', 'head', 'hair', 'hair', 'lips', 'Eyes', 'eyes', 'eyebrows', 'nose', 'Nose']


In [100]:
print(adjectives)

['short', 'oval', 'curly', 'long', 'bold', 'tired', 'sharp', 'solid', 'narrow', 'long']


In [101]:
# # Combine the adjectives and keywords into phrases
phrases = [" ".join(adj_keyword) for adj_keyword in zip(adjectives, keywords) if adj_keyword]
print("Original list: ", phrases)

# for phrase in phrases:
#     if 'hair' in phrase:
#         phrases.remove(phrase)
#         break
# print(phrases)

latest_phrases = {}

for i, phrase in enumerate(phrases):
    for keyword in keyword_list:
        if keyword in phrase.lower():
            if keyword not in latest_phrases or i > latest_phrases[keyword][1]:
                latest_phrases[keyword] = (phrase, i)

latest_phrases_list = [x[0] for x in sorted(latest_phrases.values(), key=lambda x: x[1], reverse=True)]
print("Final list: ", latest_phrases_list)

Original list:  ['short hair', 'oval head', 'curly hair', 'long hair', 'bold lips', 'tired Eyes', 'sharp eyes', 'solid eyebrows', 'narrow nose', 'long Nose']
Final list:  ['long Nose', 'solid eyebrows', 'sharp eyes', 'bold lips', 'long hair', 'oval head']


In [105]:
difference = set(phrases) - set(latest_phrases_list)
rejected_phrases_list = list(difference)
print("Rejected list: ", rejected_phrases_list)

Rejected list:  ['short hair', 'tired Eyes', 'curly hair', 'narrow nose']


In [103]:
# Loop through each phrase and display the corresponding image
for phrase in latest_phrases_list:
    # Get the keyword from the phrase
    keyword = phrase.split()[-1]
    # Define the path to the subfolder
    subfolder = os.path.join(image_folder, keyword)
    # Search for the image file in the subfolder
    for file in os.listdir(subfolder):
        if phrase.lower() in file.lower():
            # Combine the subfolder path and the image file name
            image_path = os.path.join(subfolder, file)
            # Display the image
            Image.open(image_path).show()
            # Print a message that the image is present for this phrase
            print(f"Image present for phrase: {phrase}")
            # Exit the loop since we found the image
            break
    else:
        # Image not found, append the phrase to the feedback file
        with open(feedback_file, "a") as file:
            file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {phrase} (image not found in {subfolder})\n")



Image present for phrase: sharp eyes
Image present for phrase: long hair
Image present for phrase: oval head


In [104]:
for phrase in rejected_phrases_list:
    # Get the keyword from the phrase
    keyword = phrase.split()[-1]
    # Define the path to the subfolder
    subfolder = os.path.join(image_folder, keyword)
    # Search for the image file in the subfolder
    image_found = False
    for file in os.listdir(subfolder):
        if phrase.lower() in file.lower():
            # Combine the subfolder path and the image file name
            image_path = os.path.join(subfolder, file)
            # Display the image
            # Image.open(image_path).show()
            # Print a message that the image is present for this phrase
            print(f"Image present for phrase: {phrase}")
            # Set the flag to True
            image_found = True
            # Exit the loop since we found the image
            break
    # If the image was not found, append the phrase to the feedback file
    if not image_found:
        with open(feedback_file, "a") as file:
            file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {phrase} (image not found in {subfolder})\n")

Image present for phrase: short hair
Image present for phrase: tired Eyes
